## Семинар 12
# Тема: Комплексная предобработка данных

Импортируем необходимые библиотеки:

In [128]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer, make_column_transformer

### 1. Загрузите датасет об автостраховании `StateFarm.csv` и вывеите первые 5 строк.

In [129]:
data = pd.read_csv("StateFarm.csv", sep=";")
data.head()

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


### 2. Выведите информацию о загруженном датафрейме и его размеры.

In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8293 entries, 0 to 8292
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer Lifetime Value        8289 non-null   float64
 1   Coverage                       8288 non-null   object 
 2   Education                      8290 non-null   object 
 3   EmploymentStatus               8288 non-null   object 
 4   Gender                         8289 non-null   object 
 5   Income                         8291 non-null   float64
 6   Monthly Premium Auto           8282 non-null   float64
 7   Months Since Last Claim        8288 non-null   float64
 8   Months Since Policy Inception  8285 non-null   float64
 9   Number of Open Complaints      8287 non-null   float64
 10  Number of Policies             8288 non-null   float64
 11  Response                       8293 non-null   object 
dtypes: float64(7), object(5)
memory usage: 777.6+ KB

In [131]:
data.shape

(8293, 12)

### 3. Выведите все дубликаты наблюдений и удалите их. Выведите размеры датафрейма без дубликатов.

In [132]:
data[data.duplicated()]

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
22,24127.504020,Basic,Bachelor,Medical Leave,M,14072.0,71.0,13.0,3.0,0.0,2.0,Yes
174,5780.182197,Basic,High School or Below,Employed,M,51066.0,74.0,23.0,0.0,0.0,8.0,No
199,6047.025160,Extended,College,Employed,M,20396.0,76.0,15.0,60.0,1.0,4.0,No
255,4772.943848,Extended,High School or Below,Retired,M,20993.0,133.0,35.0,90.0,0.0,1.0,No
305,8714.922135,Basic,College,Unemployed,F,0.0,118.0,14.0,64.0,0.0,4.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...
8266,19160.989940,Extended,Master,Employed,F,22705.0,239.0,19.0,59.0,0.0,7.0,Yes
8267,5548.031892,Basic,Bachelor,Employed,M,67798.0,69.0,1.0,78.0,0.0,9.0,Yes
8269,13770.976180,Premium,High School or Below,Employed,F,59207.0,116.0,23.0,21.0,0.0,2.0,Yes
8271,16979.797570,Basic,Bachelor,Employed,M,35095.0,73.0,25.0,56.0,0.0,2.0,Yes


In [133]:
data = data.drop_duplicates()
data.shape

(7494, 12)

### 4. Выведите количество пропусков и количество уникальных значений в каждом столбце.

In [134]:
pd.DataFrame({'Колличество пропусков' : data.isna().sum(), ' Колличество уникальных' : data.nunique()})

,Колличество пропусков,Колличество уникальных
Customer Lifetime Value,4,7300
Coverage,5,3
Education,3,5
EmploymentStatus,5,5
Gender,4,2
Income,2,5196
Monthly Premium Auto,11,198
Months Since Last Claim,5,36
Months Since Policy Inception,8,100
Number of Open Complaints,6,6


### 5. Обозначьте данные за X, а целевую переменную Response за y, преобразовав значения целевой переменной, используя LabelEncoder.

In [135]:
X = data.drop(columns = ['Response'])
y = data['Response']

In [136]:
X.head()

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0


In [137]:
y.value_counts()

,count
Response,
No,7274
Yes,220


In [138]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
pd.Series(y_encoded).value_counts()

,count
0,7274
1,220


### 6. Разбейте данные на обучающую и тестовую части при помощи train_test_split, зафиксировав random_state=42. В тестовый набор отправьте 30% наблюдений.

In [139]:
X_train, X_test, y_train, y_test=train_test_split(X, y_encoded,test_size =0.3, random_state =42)

In [140]:
X_train.shape,  X_test.shape

((5245, 11), (2249, 11))

### 7. Создайте списки названий категориальных и количественных признаков. Список названий категориальных признаков разбейте на списки номинальных и ординальных.

In [141]:
categorial_columns = X_train.select_dtypes(include = 'object').columns.tolist()
categorial_columns

['Coverage', 'Education', 'EmploymentStatus', 'Gender']

In [142]:
numerical_columns = X_train.select_dtypes(exclude = 'object').columns.tolist()
numerical_columns

['Customer Lifetime Value',
 'Income',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies']

In [143]:
for col in categorial_columns:
    print(X_train[col].value_counts())

Coverage
Basic       3191
Extended    1581
Premium      470
Name: count, dtype: int64
Education
Bachelor                1568
High School or Below    1542
College                 1509
Master                   428
Doctor                   195
Name: count, dtype: int64
EmploymentStatus
Employed         3302
Unemployed       1377
Medical Leave     250
Disabled          227
Retired            85
Name: count, dtype: int64
Gender
F    2684
M    2557
Name: count, dtype: int64


In [144]:
ordinals =['Coverage','Education']
nominals = ['EmploymentStatus', 'Gender']

### 8. Создайте конвейер для обработки количественных признаков, который будет осуществлять импутацию медианами и стандартизацию.

In [145]:
numerical_pipe = Pipeline([('imputer', SimpleImputer(strategy = 'median')),('scaler', StandardScaler())])
numerical_pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

### 9. Создайте два конвейера для обработки категориальных признаков (номинальных и ординальных), которые осуществляют импутацию пропусков самой часто встречающейся категорией и преобразуют признаки в числовые при помощи классов OneHotEncoder и OrdinalEncoder соответственно.

In [146]:
ordinal_pipe =Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')),('encoder', OrdinalEncoder())])
ordinal_pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OrdinalEncoder())])

In [147]:
nominal_pipe =Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')),('encoder', OneHotEncoder(sparse_output = False, drop='first'))])
nominal_pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder(drop='first', sparse_output=False))])

### 10. Создайте список трансформеров –трехэлементных кортежей. Первый элемент кортежа – название конвейера с преобразованиями для определенного списка названий признаков, второй элемент – собственно конвейер, и третий элемент – соответствующий список названий признаков. Создайте экземпляр класса ColumnTransformer, передав в него список трансформеров. Трансформируйте (предобработайте) обучающие и тестовые данные и выведите их в виде датафреймов.

In [148]:
transformers = [('numerical', numerical_pipe, numerical_columns),
                ('ordinal', ordinal_pipe, ordinals),
                ('nominal', nominal_pipe, nominals)]

In [149]:
transformer = ColumnTransformer(transformers = transformers)

In [150]:
X_train_processed = transformer.fit_transform(X_train)
X_test_processed = transformer.transform(X_test)

### 11. Обучите модель LogisticRegression(solver='lbfgs', max_iter=200). Оцените качество модели на тестовом наборе, выведя значения метрики accuracy.

In [151]:
model = LogisticRegression(solver='lbfgs', max_iter=200)

In [152]:
model.fit(X_train_processed, y_train)

LogisticRegression(max_iter=200)

In [153]:
y_pred = model.predict(X_test_processed)
accuracy_score(y_test, y_pred)

0.9684304135171187

## Задания для самостоятельного выполнения

1. Напишите класс Outlier, осуществляющий отбор выбросов по порогу. Добавьте в предобработку для признака Income обработку выбросов. Выбросами считайте доход равный нулю.

In [154]:
class Outlier():
 """
 Удалние выбросов по порогу
 """
 def __init__(self, threshold= 0):
    self.threshold = threshold

 def fit(self, X, y=None):
    return self
 def transform(self, X):
    return np.where(X <= self.threshold, np.nan, X)

In [155]:
numerical_columns = X_train.drop("Income", axis=1).select_dtypes(exclude = 'object').columns.tolist()
numerical_columns

['Customer Lifetime Value',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies']

In [156]:
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [157]:
income_pipe = Pipeline([
    ('outlier', Outlier(threshold=0)),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [158]:
X_train[['Income']].columns.tolist()

['Income']

In [159]:
transformers = [
    ('numerical', numerical_pipe, numerical_columns),
    ('ordinal', ordinal_pipe, ordinals),
    ('nominal', nominal_pipe, nominals),
    ('income', income_pipe, X_train[['Income']].columns.tolist())
]

In [160]:
preprocessor = ColumnTransformer(
    transformers=transformers
)

In [161]:
X_train_processed = transformer.fit_transform(X_train)
X_test_processed = transformer.transform(X_test)

In [162]:
preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Customer Lifetime Value',
                                  'Monthly Premium Auto',
                                  'Months Since Last Claim',
                                  'Months Since Policy Inception',
                                  'Number of Open Complaints',
                                  'Number of Policies']),
                                ('ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_fre...
                                 ['Coverage', 'Education']),
                                ('nominal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 ['EmploymentStatus', 'Gender']),
                                ('income',
                                 Pipeline(steps=[('outlier',
                                                  <__main__.Outlier object at 0x7d6e0c539d90>),
                                                 ('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Income'])])

2. Создайте конвеер, который будет осуществлять предоработку данных и обучение модели логистической регрессии. Обучите его на обучающих данных.

In [163]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='lbfgs', max_iter=200))
])

In [164]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Customer Lifetime Value',
                                                   'Monthly Premium Auto',
                                                   'Months Since Last Claim',
                                                   'Months Since Policy '
                                                   'Inception',
                                                   'Number of Open Complaints',
                                                   'Number of Policies']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('imputer...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  ['EmploymentStatus',
                                                   'Gender']),
                                                 ('income',
                                                  Pipeline(steps=[('outlier',
                                                                   <__main__.Outlier object at 0x7d6e0c539d90>),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Income'])])),
                ('classifier', LogisticRegression(max_iter=200))])

3. Выведите предобработанные обучающие и тестовые данные в виде датафреймов с названиями колонок.

In [165]:
feature_names = []

In [183]:
icome_columns = X_train[['Income']].columns.tolist()

In [167]:
# 1. Числовые колонки (остаются без изменений)
if 'numerical' in preprocessor.named_transformers_:
    feature_names.extend(numerical_columns)

# 2. Порядковые колонки (остаются без изменений)
if 'ordinal' in preprocessor.named_transformers_:
    feature_names.extend(ordinals)

# 3. Номинальные колонки (разворачиваются через OneHot)
if 'nominal' in preprocessor.named_transformers_:
    nominal_transformer = preprocessor.named_transformers_['nominal']
    ohe = nominal_transformer.named_steps['encoder']
    feature_names.extend(ohe.get_feature_names_out(nominals))

#4 обработка income
if 'income' in preprocessor.named_transformers_:
    feature_names.extend(icome_columns)

In [168]:
df_train_processed = pd.DataFrame(X_train_processed, columns=feature_names)
df_train_processed

,Customer Lifetime Value,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage,Education,EmploymentStatus_Employed,EmploymentStatus_Medical Leave,EmploymentStatus_Retired,EmploymentStatus_Unemployed,Gender_M,Income
0,-0.450250,-1.235054,1.237683,1.667096,-0.531612,-0.430342,-0.835331,1.0,3.0,0.0,0.0,0.0,1.0,1.0
1,0.172969,0.751358,0.620637,0.780019,-0.137829,-0.430342,2.498682,2.0,3.0,1.0,0.0,0.0,0.0,0.0
2,-0.695472,1.430492,-0.407773,0.484327,-0.818000,-0.430342,-0.835331,1.0,3.0,1.0,0.0,0.0,0.0,0.0
3,0.234603,-1.235054,-0.143325,-1.191264,0.578140,-0.430342,-0.418579,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.201856,-1.235054,5.351324,-1.191264,0.578140,0.663235,-0.835331,2.0,3.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5240,-0.444213,-0.905087,-0.642838,-0.107058,-0.674806,-0.430342,-0.001828,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5241,-0.617738,-1.235054,0.473721,0.977147,0.470745,1.756812,-0.835331,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5242,-0.810433,-0.606745,-0.907287,1.469968,-0.567411,-0.430342,-0.835331,0.0,4.0,0.0,0.0,0.0,0.0,1.0
5243,0.218200,1.054828,0.738170,-0.501315,0.076962,-0.430342,1.665179,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [169]:
df_test_processed = pd.DataFrame(X_test_processed, columns=feature_names)
df_test_processed

,Customer Lifetime Value,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage,Education,EmploymentStatus_Employed,EmploymentStatus_Medical Leave,EmploymentStatus_Retired,EmploymentStatus_Unemployed,Gender_M,Income
0,-0.422960,1.099045,-0.877903,-0.994135,0.792931,-0.430342,-0.001828,0.0,3.0,1.0,0.0,0.0,0.0,1.0
1,-0.801740,-1.235054,-0.672221,0.484327,-0.137829,-0.430342,-0.835331,0.0,3.0,0.0,0.0,0.0,1.0,1.0
2,-0.188219,0.426355,2.119177,-0.501315,-0.460015,-0.430342,-0.835331,2.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1.430435,1.175973,-0.113942,1.864224,-1.247582,-0.430342,-0.418579,1.0,2.0,1.0,0.0,0.0,0.0,1.0
4,-0.652427,-0.107208,-0.025792,1.765660,0.506543,1.756812,-0.835331,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,-0.028194,0.687055,-0.819137,1.469968,1.508900,-0.430342,-0.418579,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2245,0.246517,-1.235054,1.120151,1.864224,0.470745,-0.430342,2.498682,1.0,3.0,0.0,0.0,0.0,1.0,1.0
2246,-0.793632,-0.462358,-0.701605,0.385763,0.291752,-0.430342,-0.835331,0.0,3.0,0.0,1.0,0.0,0.0,0.0
2247,0.155477,-1.235054,-0.319624,0.878583,-0.961194,-0.430342,-0.418579,1.0,0.0,0.0,0.0,0.0,1.0,0.0


4. Оцените качество обученной модели на тестовых данных, выведя значения метрики accuracy.

In [170]:
y_pred = full_pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.9684304135171187

5. Повторите предобработку данных и обучение модели, используя make_pipeline, вместо Pipeline, и make_column_transformer, вместо ColumnTransformer.

In [178]:
income_pipe = make_pipeline(
    Outlier(threshold=0),
    SimpleImputer(strategy='median'),
    StandardScaler()
)

In [179]:
ordinal_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder()
)

In [180]:
nominal_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(sparse_output=False)
)

In [181]:
numerical_pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

In [184]:
preprocessor = make_column_transformer(
    (numerical_pipe, numerical_columns),
    (ordinal_pipe, ordinals),
    (nominal_pipe, nominals),
    (income_pipe, icome_columns)
)


In [185]:
full_pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(solver='lbfgs', max_iter=200)
)

In [186]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Customer Lifetime Value',
                                                   'Monthly Premium Auto',
                                                   'Months Since Last Claim',
                                                   'Months Since Policy '
                                                   'Inception',
                                                   'Number of Open Complaints',
                                                   'Number of Policies']),
                                                 ('pipeline-2',
                                                  Pi...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['EmploymentStatus',
                                                   'Gender']),
                                                 ('pipeline-4',
                                                  Pipeline(steps=[('outlier',
                                                                   <__main__.Outlier object at 0x7d6e0c7a4450>),
                                                                  ('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Income'])])),
                ('logisticregression', LogisticRegression(max_iter=200))])

In [187]:
y_pred = full_pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.9684304135171187